## FDR correction

In [1]:
## read in data
import pandas as pd
import numpy as np
import statsmodels.stats.multitest as smm

### Example demonstrating FDR corrections (Benjamini-Hochberg method)  
### Here using the correlation between neurophysiological activities and movie emotional intensity.

In [2]:
# read in
filepath = "/FDR correction/movie_bio_emotion.xlsx"
file = pd.read_excel(filepath)
file = file.apply(pd.to_numeric, errors='coerce')

In [3]:
bands = ['delta', 'theta', 'alpha', 'beta', 'gamma']
regions = ['F', 'C', 'P']

Col_Fe_eeg = [f'Fear_{region}{band}' for region in regions for band in bands]
Col_Ha_eeg = [f'Happy_{region}{band}' for region in regions for band in bands]

Col_Fe_add = ['Fear_Intensity', 'Fear_IBI', 'Fear_HF', 'Fear_LF']
Col_Ha_add = ['Happy_Intensity', 'Happy_IBI', 'Happy_HF', 'Happy_LF']

Col_Fe_all = Col_Fe_add + Col_Fe_eeg
Col_Ha_all = Col_Ha_add + Col_Ha_eeg

In [5]:
# correlation matrix
from scipy.stats import pearsonr

def correlation_with_pval(df):
    df = df.replace([np.inf, -np.inf], np.nan)  
    df = df.dropna(axis=0, how='any')
    df = df.dropna(axis=1, how='any')

    corr_matrix = df.corr()
    
    p_values = pd.DataFrame(np.zeros_like(corr_matrix, dtype=float), columns=df.columns, index=df.columns)

    for row in df.columns:
        for col in df.columns:
            _, p_val = pearsonr(df[row], df[col])
            p_values.loc[row, col] = p_val

    return corr_matrix, p_values

Mat_corr_Fe, Mat_p_Fe = correlation_with_pval(file[Col_Fe_all])
Mat_corr_Ha, Mat_p_Ha = correlation_with_pval(file[Col_Ha_all])

In [6]:
# FDR correction
from statsmodels.stats.multitest import multipletests

## Happy
first_column_Ha = Mat_corr_Ha.columns[0]
corr_values_Ha = Mat_corr_Ha[first_column_Ha].drop(first_column_Ha)  # Drop the self-correlation (first row)
p_values_Ha = Mat_p_Ha[first_column_Ha].drop(first_column_Ha)  # Drop the self-p-value (first row)
# Benjamini-Hochberg FDR correction for Ha
_, pvals_corrected_Ha, _, _ = multipletests(p_values_Ha, method='fdr_bh')

## Fear
first_column_Fe = Mat_corr_Fe.columns[0]
corr_values_Fe = Mat_corr_Fe[first_column_Fe].drop(first_column_Fe)  # Drop the self-correlation (first row)
p_values_Fe = Mat_p_Fe[first_column_Fe].drop(first_column_Fe)  # Drop the self-p-value (first row)
# Benjamini-Hochberg FDR correction for Fe
_, pvals_corrected_Fe, _, _ = multipletests(p_values_Fe, method='fdr_bh')

result_Ha = pd.DataFrame({
    'Correlation': corr_values_Ha.round(3),
    'p-value': p_values_Ha.round(3),
    'Corrected p-value': pvals_corrected_Ha.round(3)
}, index=corr_values_Ha.index)

result_Fe = pd.DataFrame({
    'Correlation': corr_values_Fe.round(3),
    'p-value': p_values_Fe.round(3),
    'Corrected p-value': pvals_corrected_Fe.round(3)
}, index=corr_values_Fe.index)

In [7]:
result_Fe

,Correlation,p-value,Corrected p-value
Fear_IBI,-0.199,0.008,0.013
Fear_HF,-0.405,0.000,0.000
Fear_LF,-0.016,0.838,0.838
Fear_Fdelta,-0.206,0.006,0.011
Fear_Ftheta,-0.142,0.060,0.077
Fear_Falpha,0.120,0.112,0.135
Fear_Fbeta,0.389,0.000,0.000
Fear_Fgamma,0.263,0.000,0.001
Fear_Cdelta,-0.267,0.000,0.001
Fear_Ctheta,-0.158,0.036,0.050


In [8]:
result_Ha

,Correlation,p-value,Corrected p-value
Happy_IBI,0.205,0.006,0.014
Happy_HF,0.301,0.000,0.000
Happy_LF,-0.020,0.788,0.835
Happy_Fdelta,-0.199,0.008,0.016
Happy_Ftheta,-0.150,0.047,0.078
Happy_Falpha,0.290,0.000,0.000
Happy_Fbeta,-0.026,0.734,0.826
Happy_Fgamma,-0.303,0.000,0.000
Happy_Cdelta,-0.166,0.028,0.050
Happy_Ctheta,-0.000,0.997,0.997
